IMPORT, READ_CSV

In [ ]:
#Standard imports
import pandas as pd
import string
import re
import numpy as np

#SKlearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#NLTK
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer, sent_tokenize
from nltk import WordNetLemmatizer # lemmatizer using WordNet
from nltk.corpus import wordnet # imports WordNet
from nltk import pos_tag # nltk's native part of speech tagging

In [ ]:
# import tensorflow as tf
# tf.config.set_visible_devices([], 'GPU')
# tf.debugging.set_log_device_placement(True)

# # Create some tensors and perform an operation
# a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
# b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
# c = tf.matmul(a, b)

# print(c)

In [ ]:
df = pd.read_csv('/Users/javm/Desktop/Projects/Book-Recommendations/books_with_blurbs.csv')

In [ ]:
df

TEXT COMPILATION

In [ ]:
df['Whole_Text']= df['Title'] + " " + df ['Author'] + " " + df['Publisher'] + " " + df['Blurb']

In [ ]:
train_df = df.drop(columns = ['ISBN'])

In [ ]:
train_df

In [ ]:
train_df.info()

PREPROCESSING

In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
    
        pass
    
    def fit(self, data, y = 0):
        
        return self
    
    def transform(self, data, y = 0):
        fully_normalized_corpus = data.apply(self.process_doc)
        
        return fully_normalized_corpus
        
    def process_doc(self, doc):

        wnl = WordNetLemmatizer()
        stop_words = stopwords.words('english')
        
        def pos_tagger(nltk_tag):
            if nltk_tag.startswith('J'):
                return wordnet.ADJ
            elif nltk_tag.startswith('V'):
                return wordnet.VERB
            elif nltk_tag.startswith('N'):
                return wordnet.NOUN
            elif nltk_tag.startswith('R'):
                return wordnet.ADV
            else:         
                return None

        doc_norm = [tok.lower() for tok in word_tokenize(doc) if (tok.isalpha())]

        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(doc_norm))) 
        doc_norm = [wnl.lemmatize(token, pos) for token, pos in wordnet_tagged if pos is not None]

        return " ".join(doc_norm)


In [ ]:
tp = TextPreprocessor()

In [ ]:
train_df['norm_text'] = tp.fit_transform(train_df['Whole_Text'])

In [ ]:
train_df = train_df.drop(columns = ['Whole_Text'])

In [ ]:
text = train_df['norm_text']

In [ ]:
# def clean_text(corpus):
#     cleaned_text = []
#     cleaned_text2 = []
#     corpus = re.sub("'", "", corpus)
#     corpus = re.sub("(\\d|\\W)+"," ",corpus)
#     corpus = re.sub("/[^A-Z0-9]+/ig", "",corpus)
#     corpus = corpus.replace('_','')
#     cleaned_text = [wn.lemmatize(word, pos = 'v') for word in word_tokenize(corpus.lower()) if black_txt(word)]
#     cleaned_text2 = [word for word in cleaned_text if black_txt(word)]
#     return " ".join(cleaned_text2)

In [ ]:
vectorizer = TfidfVectorizer(min_df = 3, max_df = 10, max_features = 5000)
vectors = vectorizer.fit_transform(text)

In [ ]:
vec_df = pd.DataFrame(vectors.toarray(),
                     columns=[k for k, v in sorted(vectorizer.vocabulary_.items(), 
                     key=lambda item: item[1])])

In [ ]:
vec_df

In [ ]:
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()

In [1]:
book_search = input('What are you reading?: ')

In [ ]:
search = clean_text(book_search)
search_vec = cvec.transform([search])